In [30]:
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

shapes = {
        "base": (94362233, 4),
        "valid": (1965045, 4),
        "hidden": (1964391, 4),
        "probe": (1374739, 4),
        "qual": (2749898, 3),
        "training": (99666408, 4),
        "training_no_probe": (98291669, 4)
}

valid = np.memmap('../dumps/mu/valid_mmap.dat', dtype='uint32', mode='r',
                  shape=shapes["valid"])

small = []
with open('../graphchi-cpp/data/smallnetflix_mm') as small_file:
    [small_file.readline() for i in range(3)]
    i = 0
    for line in small_file:
        data = list(map(int, line.strip().split()))
        #print(data)
        small.append([data[0] - 1, data[1] - 1, data[2]])
        # 95526 3561 3298163
        i += 1

# n_ratings = valid.shape[0]
n_ratings = len(small)
n_users = 458293
n_movies = 17770
ratings = {}

In [31]:
small[0]

[12, 0, 1]

In [ ]:
for i in range(n_users):
    ratings[i] = {}
"""
for line in valid:
    if line[0] == 0 or line[1] == 0:
        print(line)
        continue
    ratings[line[0] - 1][line[1] - 1] = line[3] 
"""
for line in small:
    ratings[line[0]][line[1]] = line[2]

user_list = np.array(list(ratings.keys()))

In [45]:
def build_vectors(user_ids):
    vecs = np.zeros((len(user_ids), n_movies * 5))
    for i, user_id in enumerate(user_ids):
        for movie_id in ratings[user_id]:
            vecs[i][5 * movie_id + ratings[user_id][movie_id]] = 1
    return vecs

In [33]:
ratings[3]

{1: 1,
 3: 3,
 5: 2,
 20: 2,
 23: 1,
 38: 2,
 40: 1,
 41: 5,
 50: 1,
 51: 3,
 53: 4,
 57: 2,
 58: 5,
 64: 3,
 71: 1,
 76: 2,
 79: 1,
 80: 1,
 83: 2,
 87: 3,
 92: 1,
 93: 1,
 95: 1,
 96: 4,
 98: 3,
 101: 3,
 105: 4,
 106: 1,
 109: 1,
 112: 1,
 114: 4,
 118: 2,
 121: 1,
 123: 1,
 135: 4,
 138: 4,
 149: 4,
 151: 1,
 153: 5,
 158: 1,
 161: 1,
 162: 1,
 163: 4,
 174: 3,
 176: 2,
 177: 2,
 183: 4,
 186: 2,
 190: 2,
 191: 2,
 192: 4,
 200: 4,
 205: 2,
 206: 2,
 214: 4,
 215: 3,
 217: 5,
 221: 4,
 228: 4,
 242: 1,
 244: 2,
 248: 4,
 249: 3,
 252: 1,
 255: 2,
 259: 1,
 263: 1,
 266: 3,
 269: 1,
 274: 2,
 277: 3,
 279: 1,
 280: 1,
 282: 3,
 285: 2,
 287: 1,
 291: 4,
 295: 3,
 303: 1,
 307: 1,
 310: 4,
 313: 1,
 315: 4,
 329: 4,
 334: 1,
 335: 1,
 337: 3,
 338: 1,
 347: 1,
 350: 3,
 351: 5,
 353: 2,
 354: 1,
 360: 4,
 363: 2,
 367: 1,
 369: 1,
 370: 2,
 373: 1,
 380: 4,
 383: 4,
 387: 1,
 388: 1,
 393: 4,
 394: 3,
 401: 1,
 405: 1,
 415: 1,
 419: 2,
 420: 1,
 421: 1,
 432: 4,
 433: 4,
 434: 4,
 4

In [54]:
# Setting the models Parameters
hiddenUnits = 50
visibleUnits = n_movies * 5
vb = tf.placeholder(tf.float32, [visibleUnits])  # Number of unique movies
hb = tf.placeholder(tf.float32, [hiddenUnits])  # Number of features were going to learn
W = tf.placeholder(tf.float32, [visibleUnits, hiddenUnits])  # Weight Matrix

# Phase 1: Input Processing
v0 = tf.placeholder("float", [None, visibleUnits])
_h0 = tf.nn.sigmoid(tf.matmul(v0, W) + hb)  # Visible layer activation
h0 = tf.nn.relu(tf.sign(_h0 - tf.random_uniform(tf.shape(_h0))))  # Gibb's Sampling

# Phase 2: Reconstruction
_v1 = tf.nn.sigmoid(tf.matmul(h0, tf.transpose(W)) + vb)  # Hidden layer activation
v1 = tf.nn.relu(tf.sign(_v1 - tf.random_uniform(tf.shape(_v1))))
h1 = tf.nn.sigmoid(tf.matmul(v1, W) + hb)

""" Set RBM Training Parameters """

# Learning rate
alpha = 1.0

# Create the gradients
w_pos_grad = tf.matmul(tf.transpose(v0), h0)
w_neg_grad = tf.matmul(tf.transpose(v1), h1)

# Calculate the Contrastive Divergence to maximize
CD = (w_pos_grad - w_neg_grad) / tf.to_float(tf.shape(v0)[0])

# Create methods to update the weights and biases
update_w = W + alpha * CD
update_vb = vb + alpha * tf.reduce_mean(v0 - v1, 0)
update_hb = hb + alpha * tf.reduce_mean(h0 - h1, 0)

# Set error function, here we use RMSE
err_sum = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(v0, v1))))
"""
# Set the error function, here we use Mean Absolute Error Function
err = v0 - v1
err_sum = tf.reduce_mean(err*err)
"""


""" Initialize our Variables with Zeroes using Numpy Library """

# Current weight
cur_w = np.zeros([visibleUnits, hiddenUnits], np.float32)

# Current visible unit biases
cur_vb = np.zeros([visibleUnits], np.float32)

# Current hidden unit biases
cur_hb = np.zeros([hiddenUnits], np.float32)

# Previous weight
prv_w = np.zeros([visibleUnits, hiddenUnits], np.float32)

# Previous visible unit biases
prv_vb = np.zeros([visibleUnits], np.float32)

# Previous hidden unit biases
prv_hb = np.zeros([hiddenUnits], np.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [40]:
user_list[0:10]

array([389601,  85143, 192940, 396554, 383307, 213706,  61081, 341463,
        75662,  84712])

In [ ]:
# Train RBM with 15 Epochs, with Each Epoch using 10 batches with size 100, After training print out the error by epoch
epochs = 15
batchsize = 100
errors = []
for i in range(epochs):
    # shuffle dataset
    np.random.shuffle(user_list)
    for start, end in zip(range(0, len(user_list), batchsize), range(batchsize, len(user_list), batchsize)):
        batch = build_vectors(user_list[start:end])
        cur_w = sess.run(update_w, feed_dict={v0: batch, W: prv_w, vb: prv_vb, hb: prv_hb})
        cur_vb = sess.run(update_vb, feed_dict={v0: batch, W: prv_w, vb: prv_vb, hb: prv_hb})
        cur_hb = sess.run(update_hb, feed_dict={v0: batch, W: prv_w, vb: prv_vb, hb: prv_hb})
        prv_w = cur_w
        prv_vb = cur_vb
        prv_hb = cur_hb
    errors.append(sess.run(err_sum, feed_dict={v0: trX, W: cur_w, vb: cur_vb, hb: cur_hb}))
    print(errors[-1])
plt.plot(errors)
plt.ylabel('Error')
plt.xlabel('Epoch')
plt.show()